# Save Point Cloud Data - H2O dataset 

In [ ]:
import os
import numpy as np
import cv2
import open3d as o3d
import tqdm

In [ ]:
def read_text(text_path, offset=0, half=0):
    with open(text_path, 'r') as txt_file:
        data = txt_file.readline().split(" ")
        data = list(filter(lambda x: x != "", data))

    if half:
        data_list = np.array(data)[offset:half].tolist(
        ) + np.array(data)[half+offset:].tolist()
        return np.array(data_list).reshape((-1, 3)).astype(np.float32)
    else:
        return np.array(data)[offset:].reshape((-1, 3)).astype(np.float32)

def cal_points2d(depth_img):
    points_2d = []
    for u in np.arange(depth_img.shape[1]):
        for v in np.arange(depth_img.shape[0]):
            points_2d.append(np.array([u, v]).reshape((1, -1)))
    points_2d = np.vstack(points_2d)
    return points_2d


def get_xyz_from_depth(depth_img, cam_mtx, points_2d, refl_img=None, ir=False):
    # The valid range is between 50cm and 3.86 m (NFOV unbinned), 5.46 m NFOV 2x2 binned (SW)
    Z_MIN = 0.5  # 0.25
    if ir:
        Z_MAX = 5.00
    else:
        Z_MAX = 3.86  # 2.88

    fx = cam_mtx[0, 0]
    fy = cam_mtx[1, 1]
    cx = cam_mtx[0, 2]
    cy = cam_mtx[1, 2]
    do_colors = refl_img is not None

    x_vec = (points_2d[:, 0]-cx)*(depth_img.T).reshape(1, -1) / fx
    y_vec = (points_2d[:, 1]-cy)*(depth_img.T).reshape(1, -1) / fy
    z_vec = (depth_img.T).reshape(1, -1)
    mask = np.where((Z_MIN < z_vec[0]) & (Z_MAX > z_vec[0]))
    points = np.array([x_vec[0], y_vec[0], z_vec[0]]).T

    if do_colors:
        color = (refl_img/255.).astype(np.float64)
        color = np.swapaxes(color, 0, 1)
        colors = color.reshape((-1, 3))
        colors = np.flip(colors, 1)
        mask_color = np.where((Z_MIN < z_vec[0]) & (Z_MAX > z_vec[0]))

        return points[mask], colors[mask]
    else:
        return points[mask]


def compute_pcloud(depth_img, refl_img, cam, points_2d, scale_to_m=False, rgb=False, ir=False):

    if scale_to_m:
        depth_img = depth_img.astype(float)/1000.

    # if (rgb==True):
    #    points, colors = get_xyz_from_depth(depth_img, cam.new_cam_mtx, points_2d,refl_img,True)
    # else:
    points, colors = get_xyz_from_depth(
        depth_img, cam, points_2d, refl_img, ir)
    pcloud = o3d.geometry.PointCloud()
    pcloud.points = o3d.utility.Vector3dVector(points)
    pcloud.colors = o3d.utility.Vector3dVector(colors)
    return pcloud


def get_pointcloud(depth_path, refl_path, calib, points_2d, rgb=0, ir=False):
    depth_img = cv2.imread(
        depth_path, cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)
    refl_img = cv2.imread(refl_path, cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)
    if ir:
        refl_img = cv2.cvtColor(refl_img, cv2.COLOR_GRAY2RGB)
    pcloud = compute_pcloud(depth_img, refl_img, calib,
                            points_2d, scale_to_m=True, ir=ir)
    #pcloud, ind0 = pcloud.remove_radius_outlier(nb_points = NB_POINTS, radius = RADIOUS)
    return pcloud

In [ ]:
# Save point cloud data for H2O dataset
root_dir = '/home/local/KHQ/sri.hegde/kitware/activity_recognition/datasets/h2o/'
train_file = os.path.join(root_dir, 'label_split/pose_train.txt')

with open(train_file) as f:
    lines = f.readlines()
    frame_list = [os.path.join(root_dir, line.strip()) for line in lines]

depth_img = cv2.imread(os.path.join('/'.join(frame_list[0].split('/')[:-2]), 'depth/000000.png'), cv2.IMREAD_ANYDEPTH)
point2d = cal_points2d(depth_img)

# print(len(frame_list))
for fname in tqdm.tqdm(frame_list):
    frm_name = os.path.splitext(os.path.basename(fname))[0]
    undist_depth = os.path.join('/'.join(fname.split('/')[:-2]), 'depth',f'{frm_name}.png')
    undist_rgb = fname
    calib_path = os.path.join('/'.join(fname.split('/')[:-2]), 'cam_intrinsics.txt')
    cam_fx, cam_fy, cam_cx, cam_cy, _, _ = read_text(calib_path).reshape((6))
    color_calib = np.array([[cam_fx, 0, cam_cx], [0, cam_fy, cam_cy], [0, 0, 1]])
    
    pcloud = get_pointcloud(undist_depth, undist_rgb, color_calib, point2d)
    cam_pose_path = os.path.join('/'.join(fname.split('/')[:-2]), 'cam_pose',f'{frm_name}.txt')
    with open(cam_pose_path, 'r') as txt_file:
        data = txt_file.readline().split(" ")
        data = list(filter(lambda x: x != "", data))

    cam_pose = np.array(data).astype(np.float).reshape((4, 4))
    pcloud.transform(cam_pose)
    pcloud = np.asarray(pcloud.points)

    pc_path='/'.join(fname.split('/')[:-2] + ['pc'])
    if not os.path.exists(pc_path):
        os.makedirs(pc_path)
    pc_path = os.path.join(pc_path,f'{frm_name}.txt')
#     print(pc_path)
    np.savetxt(pc_path, pcloud)